In [1]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.linear_model import Ridge
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error, r2_score, median_absolute_error
import pandas as pd
import numpy as np
from skopt.space import Real, Integer, Categorical
from skopt import BayesSearchCV
from data import df
import pickle
from sklearn import set_config

In [2]:
df.head()

,description,result_timestamp,laeq,hour,month,day_of_week,night_of_week,lat,lon,lc_dwptemp,lc_rainin,lc_dailyrain,lc_windspeed,lc_temp_qcl0,lc_temp_qcl1,lc_temp_qcl2,lc_temp_qcl3,count
0,MP 01: Naamsestraat 35 Maxim,2022-04-01 00:10:00,61.989333,0,4,Friday,Thursday,50.87725,4.700713,1.35,0.0,0.0,0.01,3.34,3.34,3.283,3.238540,13
1,MP 01: Naamsestraat 35 Maxim,2022-04-01 00:20:00,62.095500,0,4,Friday,Thursday,50.87725,4.700713,1.42,0.0,0.0,0.00,3.38,3.38,3.323,3.294944,13
2,MP 01: Naamsestraat 35 Maxim,2022-04-01 00:30:00,63.908667,0,4,Friday,Thursday,50.87725,4.700713,1.35,0.0,0.0,0.09,3.33,3.33,3.273,3.385025,13
3,MP 01: Naamsestraat 35 Maxim,2022-04-01 00:40:00,65.064833,0,4,Friday,Thursday,50.87725,4.700713,1.28,0.0,0.0,0.04,3.28,3.28,3.223,3.279090,13
4,MP 01: Naamsestraat 35 Maxim,2022-04-01 00:50:00,67.710833,0,4,Friday,Thursday,50.87725,4.700713,1.24,0.0,0.0,0.03,3.23,3.23,3.173,3.201228,13


In [3]:
df.result_timestamp = pd.to_datetime(df.result_timestamp)

In [4]:
#Drop the features we have decided not to use in the models
df = df.drop(['lc_temp_qcl0', 'lc_temp_qcl1', 'lc_temp_qcl2', 'lc_temp_qcl3', 'result_timestamp','lat','lon'], axis=1)

In [5]:
#Create a dataframe for the features and one for the target variable, laeq
X = df.loc[:, df.columns != 'laeq']
y = df.loc[:,'laeq']

In [6]:
#Create test and training sets
X_train, X_val, y_train, y_val = train_test_split(X, y, train_size=0.8, random_state=12)

In [7]:
# Define the preprocessing steps we want to include in our pipeline
numerical_cols = ['lc_dwptemp', 'lc_rainin', 'lc_dailyrain', 'lc_windspeed', 'count']

categorical_cols = ['description', 'hour', 'month', 'day_of_week', 'night_of_week']

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

preprocessor = ColumnTransformer([
    ('cat', categorical_transformer, categorical_cols),
    ('num', numerical_transformer, numerical_cols)
],
remainder='passthrough',
verbose_feature_names_out = False,
).set_output(transform="pandas")

In [8]:
#Create the pipeline
pipe = Pipeline([
    ('preprocess', preprocessor),
    ('classifier', [])
])

In [9]:
#Define the different models we want to compare
grid = [{'classifier':[RandomForestRegressor(), Ridge(), SVR()]}]

In [10]:
#Define the grid search
gridSearch = GridSearchCV(pipe, grid, scoring='neg_root_mean_squared_error', n_jobs = -1, verbose=1, cv=3)

In [11]:
#Fit the search to our training data
gridSearch.fit(X_train, y_train)

Fitting 3 folds for each of 3 candidates, totalling 9 fits


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('preprocess',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('cat',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer(strategy='most_frequent')),
                                                                                         ('onehot',
                                                                                          OneHotEncoder(handle_unknown='ignore',
                                                                                                        sparse_output=False))]),
                                                                         ['description',
                                                                          'hour',
                                                                          'month',
                                                                          'day_of_week',
                                                                          'night_of_week']),
                                                                        ('num',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer()),
                                                                                         ('scaler',
                                                                                          StandardScaler())]),
                                                                         ['lc_dwptemp',
                                                                          'lc_rainin',
                                                                          'lc_dailyrain',
                                                                          'lc_windspeed',
                                                                          'count'])],
                                                          verbose_feature_names_out=False)),
                                       ('classifier', [])]),
             n_jobs=-1,
             param_grid=[{'classifier': [RandomForestRegressor(), Ridge(),
                                         SVR()]}],
             scoring='neg_root_mean_squared_error', verbose=1)

In [12]:
#The best model as selected by the grid search
print(gridSearch.best_score_)
gridSearch.best_estimator_

-2.5386986547323938


Pipeline(steps=[('preprocess',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse_output=False))]),
                                                  ['description', 'hour',
                                                   'month', 'day_of_week',
                                                   'night_of_week']),
                                                 ('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['lc_dwptemp', 'lc_rainin',
                                                   'lc_dailyrain',
                                                   'lc_windspeed', 'count'])],
                                   verbose_feature_names_out=False)),
                ('classifier', RandomForestRegressor())])

In [13]:
#Use the best model to predict on the test data
preds = gridSearch.best_estimator_.predict(X_val)

In [14]:
#Calculate performance measures
mse = mean_squared_error(y_val, preds, squared=True)
r2 = r2_score(y_val,preds)
mae = median_absolute_error(y_val, preds)

In [15]:
#Define the parameters we want to tune and the ranges over which to do so

n_estimators = Integer(25, 200)
max_features = ['sqrt', 'log2', None]
max_depth = Integer(2,30)
min_samples_split = Integer(2,15)
bootstrap = [True, False]

In [16]:
#Define the parameter grid
param_grid = {
    'classifier__n_estimators': n_estimators,
    'classifier__max_features': max_features,
    'classifier__max_depth': max_depth,
    'classifier__min_samples_split': min_samples_split,
    'classifier__bootstrap': bootstrap,
}

In [17]:
#Initialise the Bayes search
bayesSearch = BayesSearchCV(gridSearch.best_estimator_, search_spaces=param_grid, scoring='neg_root_mean_squared_error', n_iter=20, n_jobs=-1)

In [18]:
#Fit it to the training data
bayesSearch.fit(X_train, y_train)

BayesSearchCV(estimator=Pipeline(steps=[('preprocess',
                                         ColumnTransformer(remainder='passthrough',
                                                           transformers=[('cat',
                                                                          Pipeline(steps=[('imputer',
                                                                                           SimpleImputer(strategy='most_frequent')),
                                                                                          ('onehot',
                                                                                           OneHotEncoder(handle_unknown='ignore',
                                                                                                         sparse_output=False))]),
                                                                          ['description',
                                                                           'hour',
                                                                           'month',
                                                                           'day_of_week',
                                                                           'night_of_week']),
                                                                         ('num',
                                                                          Pipeline(steps=[('impute...
              search_spaces={'classifier__bootstrap': [True, False],
                             'classifier__max_depth': Integer(low=2, high=30, prior='uniform', transform='normalize'),
                             'classifier__max_features': ['sqrt', 'log2', None],
                             'classifier__min_samples_split': Integer(low=2, high=15, prior='uniform', transform='normalize'),
                             'classifier__n_estimators': Integer(low=25, high=200, prior='uniform', transform='normalize')})

In [19]:
#View the best score and the corresponding parameters
print(bayesSearch.best_score_)
bayesSearch.best_params_

-2.4684047012187187


OrderedDict([('classifier__bootstrap', True),
             ('classifier__max_depth', 30),
             ('classifier__max_features', None),
             ('classifier__min_samples_split', 2),
             ('classifier__n_estimators', 200)])

In [20]:
#Predict on the test set using the optimised model
preds_opt = bayesSearch.predict(X_val)

In [21]:
#Calculate performance measures
mse_opt = mean_squared_error(y_val, preds_opt, squared=True)
r2_opt = r2_score(y_val,preds_opt)
mae_opt = median_absolute_error(y_val, preds_opt)

In [22]:
#Compare performance measures between tuned and untuned model
data = {'grid':[gridSearch.best_score_,mse,r2,mae], 'bayes':[bayesSearch.best_score_,mse_opt,r2_opt,mae_opt]}
pd.DataFrame(data=data, index=['-rmse','mse','r2','mae'])

,grid,bayes
-rmse,-2.538699,-2.468405
mse,5.713239,5.690139
r2,0.867698,0.868233
mae,1.229805,1.222632


In [25]:
filename = 'model_to_use.pkl'
pickle.dump(bayesSearch.best_estimator_, open(filename, 'wb'))